In [5]:
!git clone https://github.com/BogdanNV/Model_tts.git

Cloning into 'Model_tts'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 71 (delta 8), reused 71 (delta 8), pack-reused 0
Receiving objects: 100% (71/71), 3.74 MiB | 7.11 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [1]:
import os
os.chdir('/content/Model_tts')
!pip install -r  requirements.txt

In [3]:
os.chdir('/content/Model_tts/monotonic_align')
%mkdir monotonic_align
!python setup.py build_ext --inplace

Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:384: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/Model_tts/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

os.chdir('/content/Model_tts')

import commons
import utils
from data_utils import (
    TextAudioLoader,
    TextAudioCollate,
    TextAudioSpeakerLoader,
    TextAudioSpeakerCollate,
)
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [5]:

device = torch.device("cpu")

In [6]:
model_path = 'G_*.pth' # путь до весов модели 
config_path = 'config.json' # путь до конфига

hps = utils.get_hparams_from_file(config_path)

In [7]:
if hps.model.use_mel_posterior_encoder == True:
    print("Using mel posterior encoder for VITS2")
    posterior_channels = 80  # vits2
    hps.data.use_mel_posterior_encoder = True
else:
    print("Using lin posterior encoder for VITS1")
    posterior_channels = hps.data.filter_length // 2 + 1
    hps.data.use_mel_posterior_encoder = False

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.n_mel_channels,
    None,
    n_speakers=hps.data.n_speakers,
    **hps.model,
).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint(model_path, net_g, None)

Using mel posterior encoder for VITS2


In [8]:

def get_audio(model, text, output_wav_path=None):
    from scipy.io.wavfile import write

    stn_tst = get_text(text, hps)
    with torch.no_grad():
        x_tst = stn_tst.to(device).unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
        #sid = torch.LongTensor([int(sid)]).to(device)
        audio = (
            model.infer(
                x_tst,
                x_tst_lengths,
                #sid=sid,
                noise_scale=0.667,
                noise_scale_w=0.8,
                length_scale=1,
            )[0][0, 0]
            .data.cpu()
            .float()
            .numpy()
        )
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
    
    if output_wav_path:
        write(data=audio, rate=hps.data.sampling_rate, filename=output_wav_path)

In [ ]:
os.chdir('content')

In [10]:
text = """Милиционеры всегда такие строгие!"""
get_audio(net_g, text)